In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

categories = pd.read_csv("../../data/categories.csv")
cities = pd.read_csv("../../data/cities.csv")
customers = pd.read_csv("../../data/customers.csv")
products = pd.read_csv("../../data/products.csv")
employees = pd.read_csv("../../data/employees.csv")
countries = pd.read_csv("../../data/countries.csv")
sales = pd.read_csv("../../data/sales.csv")

# Ejercicio 1
El campo TotalPrice en la tabla sales no tiene valores válidos. Utilizando la información de precios de la tabla products, calcula el valor real de la venta para cada registro y almacena en una nueva columna

Utiliza la siguiente fórmula:

TotalPriceCalculated=(Quantity×UnitPrice)×(1−Discount)

In [2]:
# Paso 1: unir ventas con precio del producto
df = sales.merge(products[['ProductID', 'Price']], on='ProductID', how='left')

# Paso 2: calcular el precio total con descuento aplicado
df['TotalPriceCalculated'] = (df['Quantity'] * df['Price']) * (1 - df['Discount'])

df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,TotalPriceCalculated
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,309.63590
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,437.82200
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,1896.44160
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,1236.01384
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,719.80200


# Ejercicio 2

## Detecta los outliers en la columna de ventas totales (TotalPriceCalculated)

Utilizando el criterio del rango intercuartílico (IQR). Luego, crea una nueva columna llamada IsOutlier que tenga el valor 1 si el registro es un outlier y 0 en caso contrario. ¿Cuántos outliers se detectaron?


In [4]:
Q1 = df['TotalPriceCalculated'].quantile(0.25)
Q3 = df['TotalPriceCalculated'].quantile(0.75)
IQR = Q3 - Q1

print(Q1, Q3, IQR)

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

df['IsOutlier'] = ((df['TotalPriceCalculated'] < limite_inferior) |
                   (df['TotalPriceCalculated'] > limite_superior)).astype(int)

cantidad_outliers = df['IsOutlier'].sum()
print(f"Cantidad de outliers detectados: {cantidad_outliers}")

df.head()


176.9361 982.1564999999999 805.2203999999999
Cantidad de outliers detectados: 48217


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,TotalPriceCalculated,IsOutlier
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,309.63590,0
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,437.82200,0
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,1896.44160,0
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,1236.01384,0
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,719.80200,0



# Ejercicio 3

A partir de la columna SalesDate, crea una nueva columna que contenga únicamente la hora de la venta.

Luego, identifica en qué hora del día se concentran más ventas totales (TotalPriceCalculated).

¿La empresa vende más durante los días de semana o en el fin de semana? Utiliza la columna SalesDate para identificar el día de la semana de cada venta, clasifica los registros como Entre semana o Fin de semana, y compara el total de ventas (TotalPriceCalculated) entre ambos grupos.


In [5]:
# Ensure that SalesDate is in datetime format
df['SalesDate'] = pd.to_datetime(df['SalesDate'])

# Create a new column with the hour (excluding date)
df['SaleHour'] = df['SalesDate'].dt.hour

# Group by hour and sum TotalPriceCalculated
sales_by_hour = df.groupby('SaleHour')['TotalPriceCalculated'].sum().reset_index()

# Get the hour with the highest sales total
peak_hour_sales = sales_by_hour.loc[sales_by_hour['TotalPriceCalculated'].idxmax()]
print(peak_hour_sales)

# Extract day of the week (0 = Monday, 6 = Sunday)
df['DayOfWeek'] = df['SalesDate'].dt.dayofweek

# Classify as Weekday or Weekend
df['DayType'] = df['DayOfWeek'].apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')

# Sum total sales by day type
sales_by_daytype = df.groupby('DayType')['TotalPriceCalculated'].sum().reset_index()
print(sales_by_daytype)

SaleHour                1.600000e+01
TotalPriceCalculated    1.790144e+08
Name: 16, dtype: float64
   DayType  TotalPriceCalculated
0  Weekday          3.123405e+09
1  Weekend          1.192863e+09


Como parte del proceso de feature engineering, en el mismo df que vienes trabajando, calcula dos nuevas columnas en el dataset de ventas:
La edad del empleado al momento de su contratación y años de experiencia al momento de realizar cada venta.
Utiliza las columnas BirthDate, HireDate (de la tabla employees) y SalesDate (de la tabla sales). Asegúrate de trabajar con fechas en formato adecuado.


In [6]:
# Me aseguro el formato datetime
employees['BirthDate'] = pd.to_datetime(employees['BirthDate'])
employees['HireDate'] = pd.to_datetime(employees['HireDate'])
df['SalesDate'] = pd.to_datetime(df['SalesDate'])  # por las dudas

# Merge usando los nombres correctos
df = df.merge(
    employees[['EmployeeID', 'BirthDate', 'HireDate']],
    left_on='SalesPersonID',
    right_on='EmployeeID',
    how='left'
)

# Calcular edad al momento de contratación
df['AgeAtHire'] = (df['HireDate'] - df['BirthDate']).dt.days

# Calcular años de experiencia al momento de la venta
df['YearsExperience'] = (df['SalesDate'] - df['HireDate']).dt.days
